## Feature Scaling and Normalization - Lab

## Introduction
In this lab, you'll practice your feature scaling and normalization skills!

## Objectives
You will be able to:
* Implement min-max scaling, mean-normalization, log normalization and unit vector normalization in python
* Identify appropriate normalization and scaling techniques for given dataset

## Back to our Boston Housing data

Let's import our Boston Housing data. Remember we categorized two variables and deleted the "NOX" (nitride oxide concentration) variable because it was highly correlated with two other features.

In [1]:
import pandas as pd
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)

# first, create bins for based on the values observed. 5 values will result in 4 bins
bins = [0, 3, 4 , 5, 24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 5 values will result in 4 bins
bins = [0, 250, 300, 360, 460, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)
boston_features = boston_features.drop("NOX",axis=1)

## Look at the histograms for the continuous variables

In [4]:
boston_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 19 columns):
CRIM              506 non-null float64
ZN                506 non-null float64
INDUS             506 non-null float64
CHAS              506 non-null float64
RM                506 non-null float64
AGE               506 non-null float64
DIS               506 non-null float64
PTRATIO           506 non-null float64
B                 506 non-null float64
LSTAT             506 non-null float64
RAD_(0, 3]        506 non-null uint8
RAD_(3, 4]        506 non-null uint8
RAD_(4, 5]        506 non-null uint8
RAD_(5, 24]       506 non-null uint8
TAX_(0, 250]      506 non-null uint8
TAX_(250, 300]    506 non-null uint8
TAX_(300, 360]    506 non-null uint8
TAX_(360, 460]    506 non-null uint8
TAX_(460, 712]    506 non-null uint8
dtypes: float64(10), uint8(9)
memory usage: 44.1 KB


In [7]:
boston_continuous_features = boston_features.select_dtypes(include = "float64")
boston_con

CRIM    ZN  INDUS  CHAS     RM    AGE     DIS  PTRATIO       B  LSTAT
0     0.00632  18.0   2.31   0.0  6.575   65.2  4.0900     15.3  396.90   4.98
1     0.02731   0.0   7.07   0.0  6.421   78.9  4.9671     17.8  396.90   9.14
2     0.02729   0.0   7.07   0.0  7.185   61.1  4.9671     17.8  392.83   4.03
3     0.03237   0.0   2.18   0.0  6.998   45.8  6.0622     18.7  394.63   2.94
4     0.06905   0.0   2.18   0.0  7.147   54.2  6.0622     18.7  396.90   5.33
5     0.02985   0.0   2.18   0.0  6.430   58.7  6.0622     18.7  394.12   5.21
6     0.08829  12.5   7.87   0.0  6.012   66.6  5.5605     15.2  395.60  12.43
7     0.14455  12.5   7.87   0.0  6.172   96.1  5.9505     15.2  396.90  19.15
8     0.21124  12.5   7.87   0.0  5.631  100.0  6.0821     15.2  386.63  29.93
9     0.17004  12.5   7.87   0.0  6.004   85.9  6.5921     15.2  386.71  17.10
10    0.22489  12.5   7.87   0.0  6.377   94.3  6.3467     15.2  392.52  20.45
11    0.11747  12.5   7.87   0.0  6.009   82.9  6.2267     15.2  396.90  13.27
12    0.09378  12.5   7.87   0.0  5.889   39.0  5.4509     15.2  390.50  15.71
13    0.62976   0.0   8.14   0.0  5.949   61.8  4.7075     21.0  396.90   8.26
14    0.63796   0.0   8.14   0.0  6.096   84.5  4.4619     21.0  380.02  10.26
15    0.62739   0.0   8.14   0.0  5.834   56.5  4.4986     21.0  395.62   8.47
16    1.05393   0.0   8.14   0.0  5.935   29.3  4.4986     21.0  386.85   6.58
17    0.78420   0.0   8.14   0.0  5.990   81.7  4.2579     21.0  386.75  14.67
18    0.80271   0.0   8.14   0.0  5.456   36.6  3.7965     21.0  288.99  11.69
19    0.72580   0.0   8.14   0.0  5.727   69.5  3.7965     21.0  390.95  11.28
20    1.25179   0.0   8.14   0.0  5.570   98.1  3.7979     21.0  376.57  21.02
21    0.85204   0.0   8.14   0.0  5.965   89.2  4.0123     21.0  392.53  13.83
22    1.23247   0.0   8.14   0.0  6.142   91.7  3.9769     21.0  396.90  18.72
23    0.98843   0.0   8.14   0.0  5.813  100.0  4.0952     21.0  394.54  19.88
24    0.75026   0.0   8.14   0.0  5.924   94.1  4.3996     21.0  394.33  16.30
25    0.84054   0.0   8.14   0.0  5.599   85.7  4.4546     21.0  303.42  16.51
26    0.67191   0.0   8.14   0.0  5.813   90.3  4.6820     21.0  376.88  14.81
27    0.95577   0.0   8.14   0.0  6.047   88.8  4.4534     21.0  306.38  17.28
28    0.77299   0.0   8.14   0.0  6.495   94.4  4.4547     21.0  387.94  12.80
29    1.00245   0.0   8.14   0.0  6.674   87.3  4.2390     21.0  380.23  11.98
..        ...   ...    ...   ...    ...    ...     ...      ...     ...    ...
476   4.87141   0.0  18.10   0.0  6.484   93.6  2.3053     20.2  396.21  18.68
477  15.02340   0.0  18.10   0.0  5.304   97.3  2.1007     20.2  349.48  24.91
478  10.23300   0.0  18.10   0.0  6.185   96.7  2.1705     20.2  379.70  18.03
479  14.33370   0.0  18.10   0.0  6.229   88.0  1.9512     20.2  383.32  13.11
480   5.82401   0.0  18.10   0.0  6.242   64.7  3.4242     20.2  396.90  10.74
481   5.70818   0.0  18.10   0.0  6.750   74.9  3.3317     20.2  393.07   7.74
482   5.73116   0.0  18.10   0.0  7.061   77.0  3.4106     20.2  395.28   7.01
483   2.81838   0.0  18.10   0.0  5.762   40.3  4.0983     20.2  392.92  10.42
484   2.37857   0.0  18.10   0.0  5.871   41.9  3.7240     20.2  370.73  13.34
485   3.67367   0.0  18.10   0.0  6.312   51.9  3.9917     20.2  388.62  10.58
486   5.69175   0.0  18.10   0.0  6.114   79.8  3.5459     20.2  392.68  14.98
487   4.83567   0.0  18.10   0.0  5.905   53.2  3.1523     20.2  388.22  11.45
488   0.15086   0.0  27.74   0.0  5.454   92.7  1.8209     20.1  395.09  18.06
489   0.18337   0.0  27.74   0.0  5.414   98.3  1.7554     20.1  344.05  23.97
490   0.20746   0.0  27.74   0.0  5.093   98.0  1.8226     20.1  318.43  29.68
491   0.10574   0.0  27.74   0.0  5.983   98.8  1.8681     20.1  390.11  18.07
492   0.11132   0.0  27.74   0.0  5.983   83.5  2.1099     20.1  396.90  13.35
493   0.17331   0.0   9.69   0.0  5.707   54.0  2.3817     19.2  396.90  12.01
494   0.27957   0.0   9.69   0.0  5.926   42.6  2.3817     

## Perform log transformations for the variables where it makes sense

Analyze the results in terms of how they improved the normality performance. What is the problem with the "ZN" variable?  

"ZN" has a lot of zeros (more than 50%!). Remember that this variable denoted: "proportion of residential land zoned for lots over 25,000 sq.ft.". It might have made sense to categorize this variable to "over 25,000 feet or not (binary variable 1/0). Now you have a zero-inflated variable which is cumbersome to work with.

## Try different types of transformations on the continuous variables

Store your final features in a dataframe `features_final`

## Summary
Great! You've now transformed your final data using feature scaling and normalization, and stored them in the `features_final` dataframe.